In [39]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
from cobra.util import create_stoichiometric_matrix
import numpy as np
import escher

In [40]:
model = read_sbml_model('iMM904.xml') # insert model HERE

In [41]:
# model.optimize()

In [42]:
model.optimize().fluxes['FCLTm']

0.0

There is no flux by default. Therefore, we have to add exchange reactions before we can analyse changes in the flux. 

In [43]:
model.add_boundary(model.metabolites.pheme_m, type='demand')
with model:
    model.objective = model.reactions.FCLTm
    print("The flux og the last reaction of the heme pathway has a flux of: "+str(model.optimize().objective_value))

The flux og the last reaction of the heme pathway has a flux of: 0.550189393939394


# The heme biosynthetic pathway

We want a lot of Protoheme C34H30FeN4O4 (id: pheme_m). Let's start by figuring out what the flux to heme currently is?

We look at the reactions heme is directely involved in... ...and then work backwards to glycine (https://www.genome.jp/kegg-bin/show_pathway?sce00860)

In [3]:
model.metabolites.pheme_m # desired product

Metabolite identifier,pheme_m
Name,Protoheme C34H30FeN4O4
Memory address,0x020caa0fe6d8
Formula,
Compartment,m
In 2 reaction(s),"HEMEOSm, FCLTm"


From glycine, eight reactions form the shorthest path (natively) in Baker's yeast. These are those reactions in cronological order. 

In [27]:
heme_1 = model.reactions.get_by_id('ALASm')
heme_2 = model.reactions.get_by_id('PPBNGS')
heme_3 = model.reactions.get_by_id('HMBS')
heme_4 = model.reactions.get_by_id('UPP3S') 
heme_12 = model.reactions.get_by_id('UPPDC1') 
heme_13 = model.reactions.get_by_id('CPPPGO')
heme_14 = model.reactions.get_by_id('PPPGOm') # second to last reaction in the heme biosynthesis
heme_15 = model.reactions.get_by_id('FCLTm')
heme_pathway = [heme_1,heme_2,heme_3,heme_4,heme_12,heme_13,heme_14,heme_15]

Let's optimise for the production of heme, and measure the fluxes of all eight reactions.

In [49]:
for i in range (0,len(heme_pathway)):
    with model:
        model.objective = heme_pathway[-1] # calculates the flux
        print(model.optimize().fluxes[heme_pathway[i].id], heme_pathway[i].id, heme_pathway[i].reaction, heme_pathway[i].name)

4.401515151515152 ALASm gly_m + h_m + succoa_m --> 5aop_m + co2_m + coa_m 5 aminolevulinate synthase
2.200757575757576 PPBNGS 2.0 5aop_c --> 2.0 h2o_c + h_c + ppbng_c Porphobilinogen synthase
0.550189393939394 HMBS h2o_c + 4.0 ppbng_c --> hmbil_c + 4.0 nh4_c Hydroxymethylbilane synthase
0.550189393939394 UPP3S hmbil_c --> h2o_c + uppg3_c Uroporphyrinogen-III synthase
0.550189393939394 UPPDC1 4.0 h_c + uppg3_c --> 4.0 co2_c + cpppg3_c Uroporphyrinogen decarboxylase (uroporphyrinogen III)
0.550189393939394 CPPPGO cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c Coproporphyrinogen oxidase (O2 required)
0.275094696969697 PPPGOm 3.0 o2_m + 2.0 pppg9_m --> 6.0 h2o_m + 2.0 ppp9_m Protoporphyrinogen oxidase  mitochondrial
0.550189393939394 FCLTm fe2_m + ppp9_m --> 2.0 h_m + pheme_m Ferrochelatase  mitochondrial
